In [1]:
import os

db_path_1 = r'C:\Users\trieu.pham\Dropbox\Projects\database\FCdata.db'
db_path_2 = r'C:\Users\PC\Dropbox\Projects\database\FCdata.db'
db_path_3 = '/Users/trieupham/Dropbox/Projects/database/FCdata.db' # Mac

directory_1 = r'C:\Users\trieu.pham\Dropbox\Projects\data_xls\detailTrans'
directory_2 = r'C:\Users\PC\Dropbox\Projects\data_xls\detailTrans'
directory_3 = '/Users/trieupham/Dropbox/Projects/data_xls/detailTrans' # Mac

# Define directories where your .xlsx files are located
directory_sales_by_category_1 = r'C:\Users\trieu.pham\Dropbox\Projects\data_xls\salebycategory'
directory_sales_by_category_2 = r'C:\Users\PC\Dropbox\Projects\data_xls\salebycategory'
directory_sales_by_category_3 = '/Users/trieupham/Dropbox/Projects/data_xls/salebycategory' # Mac

directory_sales_by_store_1 = r'C:\Users\trieu.pham\Dropbox\Projects\data_xls\salebystore'
directory_sales_by_store_2 = r'C:\Users\PC\Dropbox\Projects\data_xls\salebystore'
directory_sales_by_store_3 = '/Users/trieupham/Dropbox/Projects/data_xls/salebystore' # Mac

processed_files_log_path = r'C:\Users\trieu.pham\Dropbox\Projects\github\Python\Python\Data Analysis\Store\logs'

import os
# Check if db_path exists
if os.path.exists(db_path_1):
    db_path = db_path_1
    directory = directory_1
    print('Database path: ', db_path)
else:
    if(os.path.exists(db_path_3)):
        db_path = db_path_3
        directory = directory_3
        print('Database path: ', db_path)
    if(os.path.exists(db_path_2)):
        db_path = db_path_2
        directory = directory_2
        print('Database path: ', db_path)
# Check if directory_sales_by_category exists
if os.path.exists(directory_sales_by_category_1):
    directory_sales_by_category = directory_sales_by_category_1
else:
    if(os.path.exists(directory_sales_by_category_3)):
        directory_sales_by_category = directory_sales_by_category_3
    if(os.path.exists(directory_sales_by_category_2)):
        directory_sales_by_category = directory_sales_by_category_2

# Check if directory_sales_by_store exists
if os.path.exists(directory_sales_by_store_1):
    directory_sales_by_store = directory_sales_by_store_1
else:
    if(os.path.exists(directory_sales_by_store_3)):
        directory_sales_by_store = directory_sales_by_store_3
    if(os.path.exists(directory_sales_by_store_2)):
        directory_sales_by_store = directory_sales_by_store_2

Database path:  C:\Users\PC\Dropbox\Projects\database\FCdata.db


In [2]:
import pandas as pd
import sqlite3
import glob
import os
import logging

# Connect to the SQLite database
with sqlite3.connect(db_path) as conn:
    cursor = conn.cursor()
    # Check if the table exists, and drop it if it does
    cursor.execute('''
    DROP TABLE IF EXISTS detail_trans;
    ''')
    # Create the detail_trans table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS detail_trans (
        invoiceDate TEXT,
        invoiceCode TEXT,
        storeCode TEXT,
        employeeCode TEXT,
        employeeName TEXT,
        storeName TEXT,
        barcode TEXT,
        itemName TEXT,
        qty TEXT,
        unitPrice TEXT,
        grossSales TEXT,
        itemDiscount TEXT,
        billDiscount TEXT,
        billGrossSales TEXT,
        netSales TEXT,
        vat TEXT,
        customerCode TEXT,
        requestInvoice TEXT
    );
    ''')

    # Drop the sales_by_category table if it exists
    cursor.execute('''
    DROP TABLE IF EXISTS sales_by_category;
    ''')
    # Create the sales_by_category table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS sales_by_category (
        barcode TEXT,
        itemName TEXT,
        divisionCode TEXT,
        divisionName TEXT,
        categoryCode TEXT,
        categoryName TEXT,
        subCategoryCode TEXT,
        subCategoryName TEXT,
        itemQty TEXT,
        grossSales TEXT,
        itemDiscount TEXT,
        netSales TEXT,
        totalCostPrice TEXT
    );
    ''')
    
    # Drop the sales_by_store table if it exists
    cursor.execute('''
    DROP TABLE IF EXISTS sales_by_store;
    ''')
    # Create the sales_by_store table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS sales_by_store (
        storeCode TEXT,
        storeName TEXT,
        regionCode TEXT,
        regionName TEXT,
        billCount TEXT,
        customerCount TEXT,
        qty TEXT,
        grossSales TEXT,
        itemDiscount TEXT,
        billDiscount TEXT,
        netSales TEXT,
        vatAmount TEXT,
        totalCostPrice TEXT,
        cancelBillCount TEXT,
        exceptItemActionCount TEXT,
        dateKey TEXT
    );
    ''')
    
    conn.commit()

# Close the connection
conn.close()

In [3]:


# Setup basic logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# List all .xlsx files in the directory
xlsx_files = glob.glob(os.path.join(directory, '*.xlsx'))

# Define a function to create missing columns in the table
def add_missing_columns(conn, df):
    existing_columns = [row[1] for row in conn.execute("PRAGMA table_info(detail_trans);")]
    new_columns = [col for col in df.columns if col not in existing_columns]
    
    for col in new_columns:
        conn.execute(f'ALTER TABLE detail_trans ADD COLUMN "{col}" FLOAT;')
        logging.info(f'Added new column: {col}')

# Define a function to rename specific columns
def rename_columns(df):
    rename_map = {
        'Ví VNPAY': 'VNPAY',
        'Tiền VND': 'VND',
        'Ví MOMO': 'MOMO',
        'Ví ZaloPay': 'ZaloPay',
        'Grab Order': 'GrabOrder'
    }
    df.rename(columns=rename_map, inplace=True)
    return df


    
# Use a context manager to ensure the database connection is closed properly
with sqlite3.connect(db_path) as conn:
    for file in xlsx_files:
        try:
            # Load the data from the .xlsx file into a DataFrame
            df = pd.read_excel(file)
            
            # Rename specific columns to match the database schema
            df = rename_columns(df)
            
            # Add any missing columns to the detail_trans table
            add_missing_columns(conn, df)
            
            # Insert the data into the detail_trans table
            df.to_sql('detail_trans', conn, if_exists='append', index=False)
            
            logging.info(f'Successfully inserted data from {file}')
            
        except Exception as e:
            logging.error(f'Error inserting data from {file}: {e}')
            # Optionally, continue to the next file or handle the error as needed

# No need to explicitly commit or close the connection, as the context manager handles it
logging.info('All files have been processed.')


2024-07-06 14:59:11,076 - INFO - Added new column: VNPAY
2024-07-06 14:59:11,079 - INFO - Added new column: VND
2024-07-06 14:59:11,081 - INFO - Added new column: MOMO
2024-07-06 14:59:11,084 - INFO - Added new column: OCB-eBank
2024-07-06 14:59:11,086 - INFO - Added new column: ZaloPay
2024-07-06 14:59:11,088 - INFO - Added new column: e-Voucher
2024-07-06 14:59:11,090 - INFO - Added new column: GrabOrder
2024-07-06 14:59:11,097 - INFO - Successfully inserted data from C:\Users\PC\Dropbox\Projects\data_xls\detailTrans\detailTransExport2024-06-20__6aa76d11-a63b-4988-9ab4-c6899d8947b0.xlsx
2024-07-06 14:59:11,215 - INFO - Added new column: ShopeeFood
2024-07-06 14:59:11,221 - INFO - Successfully inserted data from C:\Users\PC\Dropbox\Projects\data_xls\detailTrans\detailTransExport2024-06-21__c7814487-c61f-43ab-a183-c850ceeaae22.xlsx
2024-07-06 14:59:11,317 - INFO - Added new column: Membership
2024-07-06 14:59:11,323 - INFO - Successfully inserted data from C:\Users\PC\Dropbox\Projects\